<a href="https://colab.research.google.com/github/ilyayath/imdb-classification/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import string
from datasets import load_dataset
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from lxml.html import fromstring
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset = load_dataset("imdb")

df_train = dataset['train'].to_pandas()
df_test = dataset['test'].to_pandas()

In [2]:
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [3]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [6]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'
    elif tag.startswith('V'):
        return 'v'
    elif tag.startswith('N'):
        return 'n'
    elif tag.startswith('R'):
        return 'r'
    else:
        return 'n'

In [7]:
def preprocessing(text):
  try:
    tree = fromstring(text)
    pure = tree.text_content()
  except Exception:
    pure = text

  pure = pure.lower()
  tokens = word_tokenize(pure)
  tagged_tokens = pos_tag(tokens)
  lemmatized_sentence = []

  for word, tag in tagged_tokens:
    if word == 'are' or word in ['is', 'am']:
        lemmatized_sentence.append(word)
    else:
        lemmatized_sentence.append(lemmatizer.lemmatize(word, get_wordnet_pos(tag)))
  flt_tokens = [
      token for token in lemmatized_sentence
      if token not in stop_words
      and token not in string.punctuation
      and token.isalpha()
  ]
  return flt_tokens

In [8]:
df_train['text'] = df_train['text'].apply(preprocessing)
df_test['text'] = df_test['text'].apply(preprocessing)

In [12]:
df_train.to_csv('train_upd.csv')
df_test.to_csv('test_upd.csv')